In [1]:
import random                        # Library to generate random numbers
import skimage                       # Library for image manipulation
import numpy as np                   # Library for array manipulation
import urllib.request                # Library to download data
import matplotlib.pyplot as plt      # Library used for plotting
from skimage import io, measure               # Module from skimage
from skimage.io import imread        # Module from skimage to read images as numpy arrays
from skimage.filters import gaussian # Module working with a gaussian filter
import pathlib                              # Library to work with file paths
import os
from ipywidgets import interactive, HBox, VBox, Layout
import ipywidgets as widgets
from skimage.morphology import binary_dilation
from skimage.segmentation import watershed
from skimage.draw import polygon
from skimage.measure import regionprops
from skimage.color import label2rgb
from skimage.filters import threshold_otsu
from skimage.morphology import binary_erosion
from skimage.morphology import binary_closing
from skimage.morphology import binary_opening
from skimage.morphology import disk
from skimage.morphology import remove_small_objects
from scipy import ndimage as ndi              # Distance Transform
from skimage.feature import peak_local_max    # Local maxima in a matrix
from skimage.segmentation import watershed    # Watershed algorithm
from skimage.filters import difference_of_gaussians
from cellpose import plot, models
import pandas as pd

In [54]:
TrtList = ['Drug10_1000nM']
RepList = ['rep2']
FOVList = ['FOV1','FOV2','FOV3','FOV4','FOV5','FOV6']
filenames =[]
for trt in TrtList:
   for rep in RepList:
       for fov in FOVList:
           filename = f"C:/Users/apurv/OneDrive/Desktop/QBio/Challenges/UQbio2024_Team5/{trt}/{rep}_{fov}.tif"
           file_dict = {
               'File':filename,
               'Treatment':trt,
               'Replica':rep,
               'FOV':fov,
           }
           filenames.append(file_dict)
print(file_dict)
print(filenames)


{'File': 'C:/Users/apurv/OneDrive/Desktop/QBio/Challenges/UQbio2024_Team5/Drug10_1000nM/rep2_FOV6.tif', 'Treatment': 'Drug10_1000nM', 'Replica': 'rep2', 'FOV': 'FOV6'}
[{'File': 'C:/Users/apurv/OneDrive/Desktop/QBio/Challenges/UQbio2024_Team5/Drug10_1000nM/rep2_FOV1.tif', 'Treatment': 'Drug10_1000nM', 'Replica': 'rep2', 'FOV': 'FOV1'}, {'File': 'C:/Users/apurv/OneDrive/Desktop/QBio/Challenges/UQbio2024_Team5/Drug10_1000nM/rep2_FOV2.tif', 'Treatment': 'Drug10_1000nM', 'Replica': 'rep2', 'FOV': 'FOV2'}, {'File': 'C:/Users/apurv/OneDrive/Desktop/QBio/Challenges/UQbio2024_Team5/Drug10_1000nM/rep2_FOV3.tif', 'Treatment': 'Drug10_1000nM', 'Replica': 'rep2', 'FOV': 'FOV3'}, {'File': 'C:/Users/apurv/OneDrive/Desktop/QBio/Challenges/UQbio2024_Team5/Drug10_1000nM/rep2_FOV4.tif', 'Treatment': 'Drug10_1000nM', 'Replica': 'rep2', 'FOV': 'FOV4'}, {'File': 'C:/Users/apurv/OneDrive/Desktop/QBio/Challenges/UQbio2024_Team5/Drug10_1000nM/rep2_FOV5.tif', 'Treatment': 'Drug10_1000nM', 'Replica': 'rep2', 'F

In [3]:
# TrtList = ['Drug5_1000nM']
# RepList = ['rep1']
# FOVList = ['FOV3']
# #filenames = dict(zip(TrtList,RepList,FOVList))
# #print(filenames)
# filenames =[]
# for trt in TrtList:
#     for rep in RepList:
#         for fov in FOVList:
#             filename = f"C:/Users/apurv/OneDrive/Desktop/QBio/Challenges/UQbio2024_Team5/{trt}/{rep}_{fov}.tif"
#             file_dict = {
#                 'File':filename,
#                 'Treatment':trt,
#                 'Replica':rep,
#                 'FOV':fov,
#             }
#             filenames.append(file_dict)
# #print(file_dict)
# print(filenames)

In [4]:
#tif_dir = 'C:/Users/apurv/OneDrive/Desktop/QBio/Challenges/UQbio2024_Team5/'
#file_dict  = {
#    'FOV2':'rep1_FOV2.tif',
#     'FOV3':'rep1_FOV3.tif',
#     'FOV4':'rep1_FOV4.tif',
#     'FOV5':'rep1_FOV5.tif',
#     'FOV6':'rep1_FOV6.tif'
    # Add more file paths as needed
#}

In [55]:
answerDict = {
    'Treatment':[],
    'Replica': [],
    'FOV': [],
    'Time':[],
    'CellCount': [],
    'CellArea': [],
    'NucleusArea':[],
    'mRNA_nucleus': [],
    'mRNA_cytosol': []
}

In [56]:
# Loop through each file
for filename in filenames:
    file_path = filename['File']
    print(file_path)
    # Load the image
    images = imread(file_path)
    
    # Segment nuclei and cytosol
    #zSlice = 0
    for zSlice in range(0,21):
        img = images[zSlice, :, :, 0]
        # RUN CELLPOSE
        model = models.Cellpose(model_type='cyto')  # model_type='cyto' or model_type='nuclei'
        masks = model.eval(img, diameter=100)[0]
        num_cells = len(np.unique(masks)) - 1
        print(num_cells)
        images_FISH = images
        img_nuc = images_FISH[zSlice, :, :, 0:2]
        use_GPU = True  # Set to True if you have a GPU - this will make it MUCH faster.
        model_nuc = models.Cellpose(gpu=use_GPU, model_type='nuclei')  # model_type='cyto' or model_type='nuclei'
        masks_nuc = model_nuc.eval(img_nuc, diameter=100, channels=[0, 1])[0]

        img_cyto = images_FISH[zSlice, :, :, 0:3]
        model_cyto = models.Cellpose(gpu=use_GPU, model_type='cyto2')  # model_type='cyto', 'cyto2' or model_type='nuclei'
        masks_cyto, flows, styles, diams = model_cyto.eval(img_cyto, diameter=100, channels=[0, 2])

        nucleus_indices = np.zeros(np.max(masks_cyto) + 1)
        for i in range(1, np.max(masks_nuc) + 1):
            posn_nucl = np.mean(np.where(masks_nuc == i), axis=1).astype(int)
            if posn_nucl[0] < masks_cyto.shape[0] and posn_nucl[1] < masks_cyto.shape[1]:
                nucleus_indices[masks_cyto[posn_nucl[0], posn_nucl[1]]] = i

        # Counting the detected particles in each cell
        list_cytosol_particles = np.zeros(np.max(masks_cyto))
        list_nuclear_particles = np.zeros(np.max(masks_cyto))

        # Selecting the color channel with RNA spots
        iChannel = 1  # which channel
        img_spots = images_FISH[zSlice, :, :, iChannel]

        # Apply a difference of Gaussians filter to the image to enhance spots
        img_spots_filtered = difference_of_gaussians(img_spots, low_sigma=1, high_sigma=5)

        threshold = 0.05
        img_spots_binary = img_spots_filtered.copy()
        img_spots_binary[img_spots_binary >= threshold] = threshold  # Making spots above the threshold equal to the threshold value.
        img_spots_binary[img_spots_binary < threshold] = 0  # Making spots below the threshold equal to 0.
        img_spots_binary[img_spots_binary != 0] = 1  # Binarization

        # Labeling. Joining pixels in "particles"
        spot_contours = measure.find_contours(img_spots_binary, 0.5)

        # Loop through the cytosol masks
        for i in range(len(spot_contours)):
            # Find the position of the particle
            posn = np.mean(spot_contours[i], axis=0).astype(int)
            if posn[0] < masks_cyto.shape[0] and posn[1] < masks_cyto.shape[1]:  # Ensure within bounds
                cell_num = masks_cyto[posn[0], posn[1]]
                if cell_num > 0 and cell_num <= num_cells:
                    list_cytosol_particles[cell_num - 1] += 1

                    # Check if the particle is also in the nucleus
                    if masks_nuc[posn[0], posn[1]] == nucleus_indices[cell_num]:
                        list_nuclear_particles[cell_num - 1] += 1

        # Fill the answer dictionary
        for cell_id in range(1, num_cells + 1):
            cell_area = np.sum(masks_cyto == cell_id)
            nuc_area = np.sum(masks_nuc ==  cell_id)
            #print(mRNA_cytosol)
            mRNA_nucleus = list_nuclear_particles[cell_id - 1]
            mRNA_cytosol = list_cytosol_particles[cell_id - 1] - mRNA_nucleus
            
            
            answerDict['Treatment'].append(filename['Treatment'])
            answerDict['Replica'].append(filename['Replica'])
            answerDict['FOV'].append(filename['FOV'])
            answerDict['Time'].append(zSlice)
            answerDict['CellCount'].append(cell_id)
            answerDict['CellArea'].append(cell_area)
            answerDict['NucleusArea'].append(nuc_area)
            answerDict['mRNA_nucleus'].append(mRNA_nucleus)
            answerDict['mRNA_cytosol'].append(mRNA_cytosol)


C:/Users/apurv/OneDrive/Desktop/QBio/Challenges/UQbio2024_Team5/Drug10_1000nM/rep2_FOV1.tif
11
11
11
11
11
11
10
10
9
9
9
9
8
8
8
6
6
6
6
6
5
C:/Users/apurv/OneDrive/Desktop/QBio/Challenges/UQbio2024_Team5/Drug10_1000nM/rep2_FOV2.tif
10
10
10
10
10
8
8
7
7
7
7
7
7
7
6
6
6
6
6
6
6
C:/Users/apurv/OneDrive/Desktop/QBio/Challenges/UQbio2024_Team5/Drug10_1000nM/rep2_FOV3.tif
12
12
12
12
12
12
10
10
9
9
9
9
9
8
8
8
7
7
7
7
7
C:/Users/apurv/OneDrive/Desktop/QBio/Challenges/UQbio2024_Team5/Drug10_1000nM/rep2_FOV4.tif
12
12
12
12
12
11
11
11
11
11
11
11
11
11
11
11
10
10
10
10
8
C:/Users/apurv/OneDrive/Desktop/QBio/Challenges/UQbio2024_Team5/Drug10_1000nM/rep2_FOV5.tif
13
13
13
13
13
12
12
12
12
12
11
11
11
11
11
11
11
11
11
11
11
C:/Users/apurv/OneDrive/Desktop/QBio/Challenges/UQbio2024_Team5/Drug10_1000nM/rep2_FOV6.tif
11
11
11
11
11
10
10
10
9
9
9
9
9
9
9
9
9
9
9
9
9


In [57]:
# Convert the dictionary to a DataFrame
df = pd.DataFrame(answerDict)

# Display the DataFrame
print(df)

          Treatment Replica   FOV  Time  CellCount  CellArea  NucleusArea  \
0     Drug10_1000nM    rep2  FOV1     0          1      7529         2769   
1     Drug10_1000nM    rep2  FOV1     0          2      7594         2702   
2     Drug10_1000nM    rep2  FOV1     0          3      7090         2692   
3     Drug10_1000nM    rep2  FOV1     0          4      7704         2463   
4     Drug10_1000nM    rep2  FOV1     0          5      6879         2960   
...             ...     ...   ...   ...        ...       ...          ...   
1206  Drug10_1000nM    rep2  FOV6    20          5      7376         2876   
1207  Drug10_1000nM    rep2  FOV6    20          6      7619         3086   
1208  Drug10_1000nM    rep2  FOV6    20          7      6756         2800   
1209  Drug10_1000nM    rep2  FOV6    20          8      8097         2627   
1210  Drug10_1000nM    rep2  FOV6    20          9      7848         2660   

      mRNA_nucleus  mRNA_cytosol  
0              0.0           0.0  
1    

In [60]:
print(df.to_markdown())

|      | Treatment     | Replica   | FOV   |   Time |   CellCount |   CellArea |   NucleusArea |   mRNA_nucleus |   mRNA_cytosol |
|-----:|:--------------|:----------|:------|-------:|------------:|-----------:|--------------:|---------------:|---------------:|
|    0 | Drug10_1000nM | rep2      | FOV1  |      0 |           1 |       7529 |          2769 |              0 |              0 |
|    1 | Drug10_1000nM | rep2      | FOV1  |      0 |           2 |       7594 |          2702 |              0 |              7 |
|    2 | Drug10_1000nM | rep2      | FOV1  |      0 |           3 |       7090 |          2692 |              0 |             11 |
|    3 | Drug10_1000nM | rep2      | FOV1  |      0 |           4 |       7704 |          2463 |              0 |              7 |
|    4 | Drug10_1000nM | rep2      | FOV1  |      0 |           5 |       6879 |          2960 |              1 |              2 |
|    5 | Drug10_1000nM | rep2      | FOV1  |      0 |           6 |       7962 |   

In [59]:
df.to_csv('C:/Users/apurv/OneDrive/Desktop/QBio/Challenges/UQbio2024_Team5/Drug_101000nMrep2Dataset.csv', index=False)